# GPTQ量化

In [1]:
import os 

os.environ["TRANSFORMERS_CACHE"] = "/opt/meituan/dolphinfs_zhangyan150/llm/transformers-tune/models"
os.environ["HF_DATASETS_OFFLINE"]= "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [2]:
!echo $TRANSFORMERS_CACHE

/opt/meituan/dolphinfs_zhangyan150/llm/transformers-tune/models


In [8]:
# 这里不量化6.7B了太大了，量化1.3B的模型 省钱嘻嘻

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset=["This is offline model", "Just some random dataset is here to quant"],
     desc_act=False,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")


In [9]:
quant_model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", 
                                                   quantization_config=quantization_config, device_map='auto')

Quantizing model.decoder.layers blocks : 100%|██████████| 24/24 [04:18<00:00, 10.76s/it]


In [10]:
# 检验准确性

quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[-1968072024, -1771280518,  1789289879,  ...,  1215579015,
                         2053435976, -1754508700],
                       [  967351451, -1785370714, -1778418564,  ...,  -878163915,
                         -110706971,  2088397576],
                       [  844485784, -1236563623,  -882554554,  ..., -1999525490,
                        -1734957961,  2020046922],
                       ...,
                       [ 2072693399, -1532532076, -1689936233,  ..., -1186452839,
                         1722255495, -1264216185],
                       [-1516574263, -1742227464, -1234700615,  ..., -2004784949,
                         1469679770,  1703914486],
                       [ -674802553, -1164479124, -1505134731,  ..., -1685665369,
                          -60266871,  -660960696]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

# AWQ量化

In [15]:
from transformers import AwqConfig, AutoConfig
from awq import AutoAWQForCausalLM

model_path = "facebook/opt-1.3b"
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}


#  修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

In [18]:
quantization_config

{'quant_method': <QuantizationMethod.AWQ: 'awq'>,
 'bits': 4,
 'group_size': 128,
 'zero_point': True,
 'version': <AWQLinearVersion.GEMM: 'gemm'>,
 'backend': <AwqBackendPackingMethod.AUTOAWQ: 'autoawq'>,
 'fuse_max_seq_len': None,
 'modules_to_fuse': None,
 'do_fuse': False}

In [ ]:
model.model.config.quantization_config = quantization_config
